In [212]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

weather = pd.read_csv('weather.txt',delimiter = ',',skipinitialspace = True)
weather.columns = weather.columns.str.strip().str.lower()

taxi = pd.read_csv('nyc_taxi_may_dec2015_newfile.csv')

bike = pd.read_csv('bike_may_dec_2015.csv')



In [213]:
taxi = taxi[taxi['Month'] != 12]
bike = bike[bike['month'] != 12]

taxi['tpep_pickup_datetime'] = pd.to_datetime(taxi['tpep_pickup_datetime'])
bike['starttime'] = pd.to_datetime(bike['starttime'])

taxi['pickup_date'] = pd.DatetimeIndex(taxi['tpep_pickup_datetime']).date
bike['startdate'] = pd.DatetimeIndex(bike['starttime']).date
taxi.tail()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,Month,Time,Day,pickup_zip,dropoff_zip,pickup_date
690117,690117,2,2015-11-30 23:56:07,2015-12-01 00:14:51,2,4.43,-73.971893,40.760406,1,N,...,3.00,0.0,0.3,21.30,11,23:56:07,30,10022.0,10007.0,2015-11-30
690118,690118,2,2015-11-30 23:56:19,2015-12-01 00:05:03,1,2.01,-73.998512,40.734722,1,N,...,2.45,0.0,0.3,12.25,11,23:56:19,30,10014.0,10007.0,2015-11-30
690119,690119,1,2015-11-30 23:57:20,2015-12-01 00:22:55,1,18.00,-73.789200,40.641834,2,N,...,10.56,0.0,0.3,63.36,11,23:57:20,30,11430.0,10009.0,2015-11-30
690120,690120,2,2015-11-30 23:59:02,2015-12-01 00:02:19,2,0.87,-73.999153,40.761059,1,N,...,0.00,0.0,0.3,5.80,11,23:59:02,30,10036.0,10036.0,2015-11-30
690121,690121,1,2015-11-30 23:59:44,2015-12-01 00:08:14,1,2.90,-73.976036,40.765671,1,N,...,0.00,0.0,0.3,11.30,11,23:59:44,30,10111.0,10029.0,2015-11-30


In [214]:
# Aggregating taxi data to daily level to get #trips
taxi_daily = taxi.groupby(['pickup_date','pickup_zip']).count()['VendorID'].reset_index()
taxi_daily = pd.DataFrame(taxi_daily)
taxi_daily.columns = ['startdate','pickup_zip', 'trip_count']

#Aggregating bike data to daily level to get #trips
bike_daily = bike.groupby(['startdate','pickup_zip']).count()['bikeid'].reset_index()
bike_daily = pd.DataFrame(bike_daily)
bike_daily.columns = ['startdate','pickup_zip','bike_trips']


In [215]:
# count no. of bike stations in a particular zip
bike_stations = bike.groupby(['pickup_zip']).agg({'start station id':pd.Series.nunique}).reset_index()
bike_stations = pd.DataFrame(bike_stations)
bike_stations.columns = ['pickup_zip','bike_stations']

bike_stations.head()

,pickup_zip,bike_stations
0,10001,5
1,10002,15
2,10003,19
3,10004,6
4,10005,5


In [216]:
#merge taxi_daily & bike_daily on zip and date
taxi_bike = pd.merge(taxi_daily, bike_daily, how ='inner',left_on = ['startdate','pickup_zip'], \
                                                                              right_on = ['startdate','pickup_zip'])
#merge bike_stations and taxi_daily on zip
taxi_bike = pd.merge(taxi_bike, bike_stations, how = 'left', left_on = 'pickup_zip', right_on = 'pickup_zip')

#<QC-check>
#taxi_bike.isnull().any()
taxi_daily.dtypes



startdate      object
pickup_zip    float64
trip_count      int64
dtype: object

In [217]:
#Merge weather and taxi_bike
weather = weather.rename(columns = {'est':'startdate', 'max temperaturef':'max_tempf', 'mean temperaturef': 'mean_tempf'})
weather['startdate'] = pd.to_datetime(weather['startdate'])
taxi_bike['startdate'] = pd.to_datetime(taxi_bike['startdate'])
taxi_bike_weather = pd.merge(taxi_bike,weather, how = 'left',left_on = 'startdate',right_on = 'startdate')

#QC-check
taxi_bike_weather.isnull().any()


startdate                    False
pickup_zip                   False
trip_count                   False
bike_trips                   False
bike_stations                False
max_tempf                    False
mean_tempf                   False
min temperaturef             False
max dew pointf               False
meandew pointf               False
min dewpointf                False
max humidity                 False
mean humidity                False
min humidity                 False
max sea level pressurein      True
mean sea level pressurein     True
min sea level pressurein      True
max visibilitymiles           True
mean visibilitymiles          True
min visibilitymiles           True
max wind speedmph             True
mean wind speedmph            True
max gust speedmph             True
precipitationin              False
cloudcover                    True
events                        True
winddirdegrees               False
dtype: bool

In [218]:
weather.isnull().any()

startdate                    False
max_tempf                    False
mean_tempf                   False
min temperaturef             False
max dew pointf               False
meandew pointf               False
min dewpointf                False
max humidity                 False
mean humidity                False
min humidity                 False
max sea level pressurein      True
mean sea level pressurein     True
min sea level pressurein      True
max visibilitymiles           True
mean visibilitymiles          True
min visibilitymiles           True
max wind speedmph             True
mean wind speedmph            True
max gust speedmph             True
precipitationin              False
cloudcover                    True
events                        True
winddirdegrees               False
dtype: bool

In [219]:
#weather data processing for modelling-convert precipitation column to numerical
taxi_bike_weather['precipitationin'] = taxi_bike_weather['precipitationin'].str.replace('T','0.001')
taxi_bike_weather['precip'] = taxi_bike_weather['precipitationin'].astype(float)
taxi_bike_weather.dtypes



startdate                    datetime64[ns]
pickup_zip                          float64
trip_count                            int64
bike_trips                            int64
bike_stations                         int64
max_tempf                             int64
mean_tempf                            int64
min temperaturef                      int64
max dew pointf                        int64
meandew pointf                        int64
min dewpointf                         int64
max humidity                          int64
mean humidity                         int64
min humidity                          int64
max sea level pressurein            float64
mean sea level pressurein           float64
min sea level pressurein            float64
max visibilitymiles                 float64
mean visibilitymiles                float64
min visibilitymiles                 float64
max wind speedmph                   float64
mean wind speedmph                  float64
max gust speedmph               

In [220]:
taxi_bike_weather['events'] = taxi_bike_weather['events'].fillna('no_event')
from collections import Counter
Counter(taxi_bike_weather['events'])

Counter({'Fog-Rain': 478, 'Rain': 1818, 'no_event': 7428})

In [221]:
#Create weekday and holiday flag
import holidays
import datetime as dt

def ny_holidays(x):
    if x in holidays.US(state = 'NY'):
        return 'Y'
    else:
        return 'N'


taxi_bike_weather['holiday'] = taxi_bike_weather['startdate'].map(ny_holidays)
Counter(taxi_bike_weather['holiday'])



Counter({'N': 9351, 'Y': 373})

In [222]:
taxi_bike_weather['dofweek'] = taxi_bike_weather['startdate'].dt.dayofweek

taxi_bike_weather.loc[(taxi_bike_weather['dofweek'] == 0) | (taxi_bike_weather['dofweek'] == 1) | \
              (taxi_bike_weather['dofweek'] == 2) | (taxi_bike_weather['dofweek'] == 3) | (taxi_bike_weather['dofweek'] == 4)\
               , 'weekday'] = 'Y'

taxi_bike_weather.loc[(taxi_bike_weather['dofweek'] == 5) | (taxi_bike_weather['dofweek'] == 6),'weekday'] = 'N'



In [223]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
sns.lmplot(x='bike_trips', y='trip_count',data=taxi_bike_weather,fit_reg=True)
sns.lmplot(x='bike_stations', y='trip_count',data=taxi_bike_weather,fit_reg=True)
sns.lmplot(x='mean_tempf', y='trip_count',data=taxi_bike_weather,fit_reg=True)
plt.show()




In [226]:
import statsmodels.formula.api as sm

result = sm.ols(formula="trip_count ~ bike_trips + mean_tempf + precip + holiday + weekday",\
                                                                                        data=taxi_bike_weather).fit()
#est.summary()
print result.summary()

                            OLS Regression Results                            
Dep. Variable:             trip_count   R-squared:                       0.559
Model:                            OLS   Adj. R-squared:                  0.559
Method:                 Least Squares   F-statistic:                     2465.
Date:                Tue, 16 Aug 2016   Prob (F-statistic):               0.00
Time:                        19:58:57   Log-Likelihood:                -48706.
No. Observations:                9724   AIC:                         9.742e+04
Df Residuals:                    9718   BIC:                         9.747e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept       44.9020      2.262     19.851   

In [227]:
taxi_bike_weather.head()

,startdate,pickup_zip,trip_count,bike_trips,bike_stations,max_tempf,mean_tempf,min temperaturef,max dew pointf,meandew pointf,...,mean wind speedmph,max gust speedmph,precipitationin,cloudcover,events,winddirdegrees,precip,holiday,dofweek,weekday
0,2015-05-01,10001.0,29,47,5,63,56,49,40,39,...,5.0,18.0,0.00,6.0,no_event,98,0.0,N,4,Y
1,2015-05-01,10002.0,43,99,15,63,56,49,40,39,...,5.0,18.0,0.00,6.0,no_event,98,0.0,N,4,Y
2,2015-05-01,10003.0,162,231,19,63,56,49,40,39,...,5.0,18.0,0.00,6.0,no_event,98,0.0,N,4,Y
3,2015-05-01,10004.0,20,33,6,63,56,49,40,39,...,5.0,18.0,0.00,6.0,no_event,98,0.0,N,4,Y
4,2015-05-01,10005.0,3,18,5,63,56,49,40,39,...,5.0,18.0,0.00,6.0,no_event,98,0.0,N,4,Y
